In [1]:
import numpy as np
import pandas as pd
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from lightgbm import LGBMRegressor

In [2]:
# 1 Physics Feature Functions

def fast_pairwise_features(X, Y, n_wec):
    X_i = X[:, :, None]
    X_j = X[:, None, :]
    Y_i = Y[:, :, None]
    Y_j = Y[:, None, :]

    dx = X_i - X_j
    dy = Y_i - Y_j

    dists = np.sqrt(dx * dx + dy * dy)
    angles = np.arctan2(dy, dx)

    iu = np.triu_indices(n_wec, k=1)

    dist_flat = dists[:, iu[0], iu[1]]
    angle_flat = angles[:, iu[0], iu[1]]

    dist_cols = [f"dist_{i}_{j}" for i in range(1, n_wec+1) for j in range(i+1, n_wec+1)]
    angle_cols = [f"angle_{i}_{j}" for i in range(1, n_wec+1) for j in range(i+1, n_wec+1)]

    return (
        pd.DataFrame(dist_flat, columns=dist_cols),
        pd.DataFrame(angle_flat, columns=angle_cols)
    )


def compute_pca_direction(X, Y):
    pts = []
    for row_x, row_y in zip(X, Y):
        pts.append(np.column_stack([row_x, row_y]))
    pts = np.vstack(pts)

    pca = PCA(n_components=2)
    pca.fit(pts)

    return pca, pca.components_[0]


def compute_alignment_features(X, Y, n_wec, wave_dir):
    N = X.shape[0]
    feats = np.zeros((N, n_wec * (n_wec - 1) // 2))

    idx = 0
    for i in range(n_wec):
        for j in range(i+1, n_wec):
            vec_x = X[:, j] - X[:, i]
            vec_y = Y[:, j] - Y[:, i]

            norm = np.sqrt(vec_x**2 + vec_y**2) + 1e-8
            u_x = vec_x / norm
            u_y = vec_y / norm

            feats[:, idx] = u_x * wave_dir[0] + u_y * wave_dir[1]
            idx += 1

    cols = [f"align_{i}_{j}" for i in range(1, n_wec+1) for j in range(i+1, n_wec+1)]
    return pd.DataFrame(feats, columns=cols)


def convex_hull_area(X, Y):
    areas = []
    for xs, ys in zip(X, Y):
        pts = np.column_stack([xs, ys])
        try:
            hull = ConvexHull(pts)
            areas.append(hull.volume)
        except:
            areas.append(0.0)
    return pd.DataFrame({"convex_hull_area": areas})


def build_physics_features(df_coords, pca_model=None, wave_dir=None):
    n_wec = df_coords.shape[1] // 2

    X = df_coords[[f"X{i}" for i in range(1, n_wec+1)]].values
    Y = df_coords[[f"Y{i}" for i in range(1, n_wec+1)]].values

    # Fit PCA only on training, reuse for test
    if pca_model is None:
        pca_model, wave_dir = compute_pca_direction(X, Y)

    # pairwise
    df_dist, df_angle = fast_pairwise_features(X, Y, n_wec)

    # spatial stats
    df_stats = pd.DataFrame({
        "min_dist": df_dist.min(axis=1),
        "mean_dist": df_dist.mean(axis=1),
        "std_dist": df_dist.std(axis=1),
        "median_dist": df_dist.median(axis=1)
    })

    # convex hull area
    df_hull = convex_hull_area(X, Y)

    # alignment
    df_align = compute_alignment_features(X, Y, n_wec, wave_dir)

    full = pd.concat([df_dist, df_angle, df_align, df_stats, df_hull], axis=1)

    return full, pca_model, wave_dir

In [3]:
# 2 Load and Split Data

df_perth = pd.read_csv("Dataset/WEC_Perth_100.csv")
df_sydney = pd.read_csv("Dataset/WEC_Sydney_100.csv")

coord_cols = [c for c in df_perth.columns if c.startswith("X") or c.startswith("Y")]

# Perth split
Xp_train_coords, Xp_test_coords, yp_train, yp_test = train_test_split(
    df_perth[coord_cols], df_perth["Total_Power"], test_size=0.2, random_state=42
)

# Sydney split
Xs_train_coords, Xs_test_coords, ys_train, ys_test = train_test_split(
    df_sydney[coord_cols], df_sydney["Total_Power"], test_size=0.2, random_state=42
)

In [4]:
# 3 Build Physics Feature Set - PCA is fit only on the training data

# Perth
Xp_train_phys, perth_pca, perth_wave = build_physics_features(Xp_train_coords)
Xp_test_phys, _, _ = build_physics_features(
    Xp_test_coords, pca_model=perth_pca, wave_dir=perth_wave
)

# Sydney
Xs_train_phys, syd_pca, syd_wave = build_physics_features(Xs_train_coords)
Xs_test_phys, _, _ = build_physics_features(
    Xs_test_coords, pca_model=syd_pca, wave_dir=syd_wave
)

In [5]:
# 4 Train 2 Models, one for Sydney and another for Perth

model_perth = LGBMRegressor(
    n_estimators=1200, learning_rate=0.01, num_leaves=64,
    subsample=0.8, colsample_bytree=0.8, random_state=42
)

model_sydney = LGBMRegressor(
    n_estimators=1200, learning_rate=0.01, num_leaves=64,
    subsample=0.8, colsample_bytree=0.8, random_state=42
)

model_perth.fit(Xp_train_phys, yp_train)
p_pred = model_perth.predict(Xp_test_phys)

model_sydney.fit(Xs_train_phys, ys_train)
s_pred = model_sydney.predict(Xs_test_phys)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.665084 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3732677
[LightGBM] [Info] Number of data points in the train set: 5821, number of used features: 14855
[LightGBM] [Info] Start training from score 7100077.682185
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.264000 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2995948
[LightGBM] [Info] Number of data points in the train set: 1854, number of used features: 14855
[LightGBM] [Info] Start training from score 7166494.074164


In [6]:
# 5 Results Summary

metrics = pd.DataFrame([
    ["Perth_100", "LightGBM",
     root_mean_squared_error(yp_test, p_pred),
     mean_absolute_error(yp_test, p_pred),
     r2_score(yp_test, p_pred)],

    ["Sydney_100", "LightGBM",
     root_mean_squared_error(ys_test, s_pred),
     mean_absolute_error(ys_test, s_pred),
     r2_score(ys_test, s_pred)]
],
columns=["Region", "Model", "RMSE", "MAE", "R²"])

display(metrics)

,Region,Model,RMSE,MAE,R²
0,Perth_100,LightGBM,36563.366481,13461.323088,0.963700
1,Sydney_100,LightGBM,14271.181253,6180.879836,0.979158


In [7]:
# 6 Relative MAE of Perth

mae_perth = mean_absolute_error(yp_test, p_pred)
relative_mae_perth = (mae_perth / yp_test.mean()) * 100

print(f"Relative MAE Perth: {relative_mae_perth}")

Relative MAE Perth: 0.18948936990308887


In [8]:
# 7 Relative MAE of Sydney

mae_sydney = mean_absolute_error(ys_test, s_pred)
relative_mae_sydney = (mae_sydney / ys_test.mean()) * 100

print(f"Relative MAE Sydney: {relative_mae_sydney}")

Relative MAE Sydney: 0.08620349040832696
